In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# [RAG 절차]

1. 문서를 읽는다
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변 얻음(렝체인 사용 가능)
    %pip install -q langchain
    (https://smith.langchain.com에서 key생성 .env에 LANGCHAIN_API_KEY로 추가)

# 0. 패키지 설치

In [ ]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

In [2]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [5]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 4.86 s
Wall time: 5.1 s


In [7]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters

## 2.1 1500토큰씩 쪼개서 읽어오기

In [8]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter
loader = Docx2txtLoader('./data/소득세법(법률)(제21065호)(20260102).docx')
# gpt-4, gpt-4o, gpt-4 turbo, gpt4o-mini, embedding모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", #토큰을 세는 방식 이름
    chunk_size=1500,             # chunk 당 토큰 수 기준
    chunk_overlap=200
    # separators = ["\n", "\n\n"]파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 쪼개면서 읽는 시간 : ", runtime)

문서를 쪼개면서 읽는 시간 :  6.768277406692505


In [10]:
len(documents) # chunk 수

180

In [17]:
# chunk 글자수
# documents[0].page_content
print([len(document.page_content) for document in documents])

[1699, 1656, 1641, 1650, 1738, 1442, 1287, 1535, 1325, 1619, 1596, 1588, 1566, 1639, 1622, 1559, 1612, 1638, 1573, 1465, 1436, 1609, 1456, 1497, 1635, 1606, 1533, 1649, 1662, 1595, 1603, 1678, 1595, 1637, 1601, 1539, 1561, 1594, 1693, 1708, 1657, 1627, 1636, 1659, 1667, 1595, 1491, 1485, 1645, 1709, 1629, 1617, 1495, 1626, 1612, 1620, 1609, 1576, 1636, 1602, 1556, 1563, 1600, 1616, 1643, 1691, 1635, 1685, 1621, 1631, 1609, 1605, 1603, 1604, 1698, 1686, 1702, 1612, 1539, 1558, 1651, 2060, 1562, 1606, 1557, 1648, 1594, 1615, 1766, 1651, 1690, 1576, 1536, 1553, 1638, 1685, 1693, 1694, 1664, 1529, 1627, 1703, 1675, 1546, 1585, 1687, 1679, 1714, 1603, 1655, 1648, 1495, 1531, 1562, 1594, 1646, 1543, 1449, 1593, 1559, 1521, 1473, 1519, 1545, 1668, 1700, 1692, 1655, 1648, 1741, 1670, 1628, 1639, 1623, 1638, 1642, 1666, 1658, 1594, 1591, 1561, 1641, 1498, 1610, 1567, 1613, 1636, 1619, 1531, 1496, 1702, 1598, 1579, 1627, 1559, 1585, 1665, 1565, 1616, 1564, 1612, 1535, 1512, 1557, 1576, 1628, 165

In [22]:
# chunk 글자수 최대값, 최소값
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

2060
955


## 2.2 1500글자 쪼개서 읽어오기

# 3. 쪼갠 문서를 임베딩 -> 벡터 베이터베이스 저장
- 임베딩 모델 : text-embedding-3-large (기본모델 :text-embedding-ada-002)
- 벡터데이터베이스(벡터 store) : chroma

In [30]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [31]:
# embed_query() 한 문자열을 임베딩 벡터로 전환한 숫자 list를 return
len(embedding.embed_query("소득세법은 어쩌구"))

3072

In [34]:
embedding_vectors = embedding.embed_documents( # 여러 문자열을 임베딩 벡터로
    [
        documents[0].page_content,
        documents[1].page_content
    ]
)

In [41]:
print(len(embedding_vectors), len(embedding_vectors[0]), len(embedding_vectors[1]))
print(embedding_vectors[0][:10])

2 3072 3072
[0.014169108122587204, -0.022027326747775078, 0.009101607836782932, 0.009438703767955303, 0.030725523829460144, 0.0165895726531744, -0.026680365204811096, 0.011428126133978367, -0.014898562803864479, 0.01189232524484396]


In [ ]:
%%time
from langchain_chroma import Chroma
# 데이터 처음 저장할 때
database = Chroma.from_documents()